In [ ]:
# Load LSTM and other required libraries

import numpy as np
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import sys



#Load text data and convert to lowercases (part of the data is available in Github)
filename = "finalconcurrent.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()


#Map unique characterss to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

#compute number of characters
n_chars = len(raw_text)
n_vocab = len(chars)


In [ ]:
#Dataset creation and LSTM mdoel development 
#Create a sequential dataset of input/output pairs (input: sequence of 10 characters, output: a single character after the input sequence)
seq_length = 10
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)

# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)



#Define LSTM model with 2 hidden layers and dropouts
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=400, batch_size=64, callbacks=callbacks_list)



In [ ]:
#implement Beam Search
#This function receives the last layer of probabilities and pattern to give the next character elements
def create_kresults(prediction,pattern,log_p):
  k=3
  #create next element [pattern,probability,next_index,result]
  n_elements=[[None for i in range(k)],[], [], []]
  pat=n_elements[0]
  log_p_next=[]
  next_index=n_elements[2]    
  result=n_elements[3]  
  for i in range(k):
      index1=numpy.argmax(prediction)
      prob1=prediction[0,index1]
      result1 = int_to_char[index1]
      next_index.append(index1)
      log_p_next.append(log_p-np.log(prob1))
      result.append(result1)
      prediction[0,index1]=0
  for i in range(k):
      pat[i]=np.array(pattern)
      pat[i]=np.append(pat[i],next_index[i])
      pat[i]=pat[i][1:len(pat[i])]

  n_elements=[pat,log_p_next,next_index,result]
  return n_elements

#Function of creating str from a pattern
def create_str(pattern):
    o=[]
    for u in range(len(pattern)):
        o.append(int_to_char[pattern[u]])
    return o
    
#Function of creating pattern from a str
def create_number(pattern):
    o=[]
    for u in range(len(pattern)):
        o.append(char_to_int[pattern[u]])
    return o     

In [ ]:
#Print next characters and their probabilities
#Define number of predictions
n_prediction=3
n=[]
for i in range(n_prediction):
  if i==0:
      x=numpy.reshape(pattern, (1, len(pattern), 1))
      x = x / float(n_vocab)
      prediction = model.predict(x, verbose=0)
      prediction1=np.copy(prediction)      
      n_elements=create_kresults(prediction1,pattern,0)
      n.append(n_elements)

  else:
      multi_n_elements=[]

      for j in range(k):    
          x=numpy.reshape(n_elements[0][j], (1, len(pattern), 1))
          x = x / float(n_vocab)
          prediction = model.predict(x, verbose=0)
          prediction1=np.copy(prediction)
          log_p1=n_elements[1][j]
          n_elements1=create_kresults(prediction1,n_elements[0][j],log_p1)
          multi_n_elements.append(n_elements1)
      r=np.concatenate((multi_n_elements[0],multi_n_elements[1],multi_n_elements[2]),axis=1)
      r=np.transpose(r)
      ordered = sorted(r, key=lambda tup:tup[1]) 
      ordered=np.reshape(ordered,(np.shape(ordered)))
      ordered=np.transpose(ordered)
      n_elements=ordered[:,0:k]   
      n.append(n_elements)
      #print
      print("next index",n_elements[2])
      print("probs",n_elements[1])
      print("result",n_elements[3])
      print("patterns",n_elements[0])

In [ ]:
#Evaluation

# -*- coding: utf-8 -*-
"""
Created on Wed Nov  4 17:46:45 2020

@author: Hamed
"""

#Generating random sequences from test database
import numpy as np
filename = "Test.txt"
raw_text1 = open(filename, 'r', encoding='utf-8').read()
raw_text1 = raw_text1.lower()
#takeout the enter character
raw_text=""
for i in range(len(raw_text1)):
    if raw_text1[i] != '\n':
        raw_text=raw_text+raw_text1[i]

# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)


#create integer data
data=[]
for char in raw_text:
    r=char_to_int[char]
    data.append(r)
    
    


#define seqin as the input to the model

def create_str(pattern):
    o=[]
    for u in range(len(pattern)):
        o.append(int_to_char[pattern[u]])
    return o
    
#Function of creating pattern from a str
def create_number(pattern):
    o=[]
    for u in range(len(pattern)):
        o.append(char_to_int[pattern[u]])
    return o  
#Function of creating str from a pattern

#AR SR BMS
stringin1=['a','r', ' ', 'r', ' ', 'r', ' ', 'r', 'q', ' ']
seqin=create_number(stringin1)
stringout1=['b','r', ' ', 'b', 'z']
pattern_outseq=create_number(stringout1)


#create all possible output sequences
import itertools
from random import sample
t=[]
for i in range(27):
    t.append(i)    
R=list(itertools.product(t,t,t,t,t,repeat=1)) 
t=[]
for i in range(len(R)):
    t.append(list(R[i]))
R=t
R=sample(R,200000)


def evaluate(c,data,seqin1):
    x1=[]
    numenator=0
    prob=0
    for o in range(len(c)):
        if c[o]+len(seqin1)+1<len(data):
            x1.append(data[c[o]+len(seqin1)+1])
        
            if x1[o]==R[i][j]:
                numenator+=1
    prob+=(numenator/len(c))
    if prob==0:
        x2=[]
        for o in range(len(c)):
            if c[o]+len(seqin1)+2<len(data):
                x2.append(data[c[o]+len(seqin1)+2])
                if x2[o]==R[i][j]:
                    numenator+=1
        prob+=(numenator/len(c))*0.5
    if prob==0:
        x3=[]
        for o in range(len(c)):
            if c[o]+len(seqin1)+3<len(data):
                x3.append(data[c[o]+len(seqin1)+3])
                if x3[o]==R[i][j]:
                    numenator+=1
        prob+=(numenator/len(c))*0.33
    if prob==0:
        x4=[]
        for o in range(len(c)):
            if c[o]+len(seqin1)+4<len(data):
                x4.append(data[c[o]+len(seqin1)+4])
                if x4[o]==R[i][j]:
                    numenator+=1
        prob+=(numenator/len(c))*0.25
    if prob==0:
        x5=[]
        for o in range(len(c)):
            if c[o]+len(seqin1)+5<len(data):
                x5.append(data[c[o]+len(seqin1)+5])
                if x5[o]==R[i][j]:
                    numenator+=1
        prob+=(numenator/len(c))*0.2
    if prob==0:
        prob=0.000001
    return -np.log(prob)




evaluation=[]
for i in range(len(R)):
    #evaluate R[i]
    eva_sequence=0
    
    for j in range(len(R[i])):
        #consider single elements of R[i][j] for evaluation
        seqin1=seqin[j:]+R[i][0:j]
        ev_element=0
        for k in range(len(seqin)):
            #look for the seqin in the database
            c=[l for l in range(len(data)) if data[l:l+len(seqin1)]==seqin1]
            if len(c)==0:
                seqin1=seqin1[1:]
            else:
                #evaluate R[i][j]if you found a match of a subsequence from the input sequence in the database
                ev_element=evaluate(c,data,seqin1)*(k) if k!=0 else evaluate(c,data,seqin1)
        eva_sequence += ev_element
    #check the existence of ' ',' ' in the sequence
    t=0
    er=0
    while t<len(R[i])-1:
        if R[i][t]==R[i][t+1]==char_to_int[' ']:
            er+=1
        t+=1
    if er>0:
        eva_sequence = er*100+eva_sequence
    evaluation.append([R[i],eva_sequence])
    if i%1000==0:
        print(i) 
evaluation=sorted(evaluation, key=lambda tup:tup[1]) 


np.save('evaluation.npy',evaluation)

np.save('evaluation1.npy',evaluation)
a=np.load('evaluation.npy')
a1=np.load('evaluation1.npy')

pattern_outseq=pattern_outseq[0:5]
#evaluate output seq of the model
R=[pattern_outseq,pattern_outseq]
evaluation1=[]
for i in range(len(R)):
    #evaluate R[i]
    eva_sequence=0
    
    for j in range(len(R[i])):
        #consider single elements of R[i][j] for evaluation
        seqin1=seqin[j:]+R[i][0:j]
        ev_element=0
        for k in range(len(seqin)):
            #look for the seqin in the database
            c=[l for l in range(len(data)) if data[l:l+len(seqin1)]==seqin1]
            if len(c)==0:
                seqin1=seqin1[1:]
            else:
                #evaluate R[i][j]if you found a match of a subsequence from the input sequence in the database
                ev_element=evaluate(c,data,seqin1)*(k) if k!=0 else evaluate(c,data,seqin1)
        eva_sequence += ev_element
    evaluation1.append([R[i],eva_sequence])
evaluation1=sorted(evaluation1, key=lambda tup:tup[1]) 


c=[]
for i in range(len(R)):
    c.append(evaluation[i][1])

# #save to excel
# import pandas as pd
# data1=pd.DataFrame(c)
# datatoexcel=pd.ExcelWriter("FromPython.xlsx")

# data1.to_excel(datatoexcel,sheet_name='Sheet1')
# datatoexcel.save()

# #read from the file
# from openpyxl import load_workbook
# xl=pd.ExcelFile("FromPython.xlsx")
# df=xl.parse("Sheet1")
# c1=df["Evaluation"]

from random import sample
f1=sample(c,200000)
f2=sample(c,200000)
f3=sample(c,200000)
f4=sample(c,200000)
f5=sample(c,200000)
f6=sample(c,100000)


c1=c+f1+f2+f3+f4+f5+f6
c1=c1.sort()
np.save("Myfile5.npy",c1)



a = np.load("Myfile5.npy")
np.percentile(a,0.6)


import matplotlib.pyplot as plt
# plt.hist(c10, bins=60)
# plt.show()


plt.figure(figsize=[10,8])
n, bins, patches = plt.hist(a, bins=60, color='#0504aa',alpha=0.7, rwidth=0.9)
plt.grid(axis='y', alpha=0.75)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.title('Possible output sequences score- Case #6' ,fontsize=20)
plt.show()